In [7]:
from typing import List, Optional

import dspy
from pydantic import Field
from dslmodel import DSLModel


# Hero Section Models

class HeroLink(DSLModel):
    label: str = Field(..., description="Label for the hero link")
    icon: Optional[str] = Field(None, description="Icon for the link")
    trailing: bool = Field(False, description="Whether to display an arrow after the link")
    to: str = Field(..., description="URL or page link")
    size: str = Field("xl", description="Size of the link")


class HeroSection(DSLModel):
    title: str = Field(..., description="Main title for the hero section")
    description: str = Field(..., description="Description for the hero section")
    headline_label: str = Field(..., description="Label for the headline")
    headline_to: str = Field(..., description="URL for the headline")
    headline_icon: str = Field(..., description="Icon for the headline")
    links: List[HeroLink] = Field(..., description="List of links in the hero section")


# Logos Section Model

class LogosSection(DSLModel):
    title: str = Field(..., description="Title for the logos section")
    icons: List[str] = Field(..., description="List of icon identifiers for logos")


# Features Section Models

class FeatureItem(DSLModel):
    title: str = Field(..., description="Title for the feature")
    description: str = Field(..., description="Description of the feature")
    icon: str = Field(..., description="Icon for the feature")


class FeaturesSection(DSLModel):
    headline: str = Field(..., description="Headline for the features section")
    title: str = Field(..., description="Title for the features section")
    description: str = Field(..., description="Description for the features section")
    items: List[FeatureItem] = Field(..., description="List of feature items")


# Pricing Section Models

class PricingPlan(DSLModel):
    title: str = Field(..., description="Title for the pricing plan")
    description: str = Field(..., description="Description of the pricing plan")
    price: str = Field(..., description="Price for the plan")
    align: str = Field("top", description="Alignment for the plan")
    highlight: Optional[bool] = Field(False, description="Whether to highlight this plan")
    scale: Optional[bool] = Field(False, description="Whether the plan scales")
    features: List[str] = Field(..., description="List of features included in this plan")


class PricingSection(DSLModel):
    headline: str = Field(..., description="Headline for the pricing section")
    title: str = Field(..., description="Title for the pricing section")
    description: str = Field(..., description="Description for the pricing section")
    plans: List[PricingPlan] = Field(..., description="List of pricing plans")


# Testimonials Section Models

class Testimonial(DSLModel):
    quote: str = Field(..., description="Quote for the testimonial")
    name: str = Field(..., description="Name of the testimonial author")
    position: str = Field(..., description="Position or title of the author")
    avatar_url: str = Field(..., description="URL for the avatar image")


class TestimonialsSection(DSLModel):
    headline: str = Field(..., description="Headline for the testimonials section")
    title: str = Field(..., description="Title for the testimonials section")
    description: str = Field(..., description="Description for the testimonials section")
    items: List[Testimonial] = Field(..., description="List of testimonials")


# FAQ Section Models

class FAQItem(DSLModel):
    question: str = Field(..., description="FAQ question")
    answer: str = Field(..., description="FAQ answer")
    default_open: bool = Field(False, description="Whether the FAQ is open by default")


class FAQSection(DSLModel):
    title: str = Field(..., description="Title for the FAQ section")
    description: str = Field(..., description="Description for the FAQ section")
    items: List[FAQItem] = Field(..., description="List of FAQ items")


# CTA Section Model

class CTASection(DSLModel):
    title: str = Field(..., description="Title for the call-to-action section")
    description: str = Field(..., description="Description for the call-to-action section")
    links: List[HeroLink] = Field(..., description="Links for the call-to-action")


# Landing Page Model

class LandingPage(DSLModel):
    hero: HeroSection = Field(..., description="Hero section")
    logos: LogosSection = Field(..., description="Logos section")
    features: FeaturesSection = Field(..., description="Features section")
    pricing: PricingSection = Field(..., description="Pricing section")
    testimonials: TestimonialsSection = Field(..., description="Testimonials section")
    faq: FAQSection = Field(..., description="FAQ section")
    cta: CTASection = Field(..., description="Call-to-action section")



In [2]:
from dslmodel import init_text

prompt = """
Create a landing page for the Marcus Campbell Ecosystem, a platform that offers various tools, services, and a community for entrepreneurs. The page should include the following:

- **Hero Section**: Title as "Empowering Entrepreneurs Worldwide", with a brief description about how Marcus Campbell Ecosystem provides essential resources to help business owners succeed. Add links for "Get Started" and "Learn More".
- **Logos Section**: Featuring icons of trusted brands like AWS, Google, and Stripe.
- **Features Section**: Highlight six key features: "Comprehensive Courses", "Expert Guidance", "Secure Platform", "Rapid Tools", "Affordable Plans", and "Scalability". Each feature should have a short description.
- **Pricing Section**: Three plans - Basic, Pro, and Premium - each with increasing levels of access and support.
- **Testimonials Section**: Include three testimonials from business owners praising the tools and community of the Marcus Campbell Ecosystem.
- **FAQ Section**: Common questions about subscriptions, cancelations, and plan upgrades.
- **CTA Section**: Encourage visitors to join the community with a sign-up link.
"""

init_text()

lp = LandingPage.from_prompt(prompt)
print(lp.to_yaml())

In [4]:
from dslmodel import render

# Updated Jinja template
template_str = """
defineProps({
{% for prop in props %}
  {{ prop.name }}: {
    type: {{ prop.type }}{% if prop.default is not none %},
    default: {{ prop.default }}{% endif %}
  }{% if not loop.last %},
  {% endif %}
{% endfor %}
});
"""

# Props to pass to the template
props = [
    {"name": "username", "type": "String", "default": "'Guest'"},
    {"name": "age", "type": "Number", "default": None},
    {"name": "isAdmin", "type": "Boolean", "default": "false"}
]

# Render the template
output = render(template_str, props=props)

print(output)


In [15]:
from pydantic import BaseModel, Field
from enum import Enum
from dslmodel import render, init_instant


# Pydantic Model for Vue Props
class VueProp(DSLModel):
    """Model for Vue props. Must use javascript types for the `type` field."""
    name: str = Field(..., description="The name of the prop.")
    type: str = Field(...,
                      description="The expected javascript type of the prop. String, Number, Boolean, Array, Object, or Function.")
    default: str | None = Field(None, description="Default value of the prop (optional).")


# Jinja template string for defining Vue props
template_str = """
defineProps({
{% for prop in props %}
  {{ prop.name }}: {
    type: {{ prop.type | class_name }}{% if prop.default is not none %},
    default: {{ prop.default }}{% endif %}
  }{% if not loop.last %},
{% endif %}
{% endfor %}
});
"""

# Define the props using the VueProp model
# props = [
#     VueProp(name="username", type=PropType.String, default="'Guest'"),
#     VueProp(name="age", type=PropType.Number, default=None),
#     VueProp(name="isAdmin", type=PropType.Boolean, default="false"),
# ]

init_instant()

props = [
    VueProp.from_prompt("username string with a default value of 'Guest'"),
    VueProp.from_prompt("age number"),
    VueProp.from_prompt("isAdmin boolean with a default value of false"),
]

# Render the Jinja template with the props
output = render(template_str, props=props)

# Print the rendered Vue <script setup> block
print(output)


In [7]:
from pydantic import BaseModel, Field
from typing import Dict
from enum import Enum
from dslmodel import render, DSLModel


class VueProp(DSLModel):
    """Model for Vue props. Must use javascript types for the `type` field."""
    name: str = Field(..., description="The name of the prop.")
    type: str = Field(...,
                      description="The expected javascript type of the prop. String, Number, Boolean, Array, Object, or Function.")
    default: str | None = Field(None, description="Default value of the prop (optional).")


class VueComponent(DSLModel):
    pascal_name: str = Field(..., description="The PascalCase name of the component.")
    kebab_name: str = Field(..., description="The kebab-case name of the component.")
    template: str = Field("<template>\n</template>", description="<template> block of the Vue component.")
    props: list[VueProp] = Field([], description="List of props for the component.")
    setup: str = Field("<script setup>\n</script>",
                       description="The <script setup> block of the component. No placeholders. Nuxt Core Team quality code only.")
    style: str = Field("<style scoped>\n</style>", description="The <style scoped> block of the component.")
    nuxt_path: str = Field("", description="The path to the component in a Nuxt project.")


component_spec = VueComponent(
    pascal_name="CardComponent",
    kebab_name="card-component",
    template="""
<template>
  <div class="card">
    <component :is="icon" class="icon" />
    <h2>{{ title }}</h2>
    <p>{{ description }}</p>
    <button @click="handleClick">Click Me</button>
  </div>
</template>
""",
    props=[
        VueProp(name="title", type="String", default="'Default title'"),
        VueProp(name="description", type="PropType", default="'Default description'"),
        VueProp(name="icon", type="String", default="'IconMarkdown'")
    ],
)

# Jinja template for rendering
template_str = """
<script setup>
defineProps({
{% for prop in props %}
  {{ prop.name }}: {
    type: {{ prop.type }},
    default: {{ prop.default }}
  }{% if not loop.last %}, {% endif %}
{% endfor %}
});
</script>

{{ template }}


"""

# Render the Vue component
output = render(template_str, props=component_spec.props,
                template=component_spec.template)
print(output)


In [8]:
from dslmodel import init_text

init_text()

inst = VueComponent.from_prompt(
    "Hello World. Must include a template that shows 'Hello, {{ name }}!'. Name must be a prop of type String with a default value of 'World'.")

In [12]:
inst.to_yaml("vue_component.yaml")
import dspy

dspy.ChainOfThought("vue_code -> nuxt_core_team_quality_code")(vue_code=str(inst)).nuxt_core_team_quality_code

In [10]:
# from pathlib import Path

# Path(inst.nuxt_path[1:]).write_text(f"{inst.setup}\n{inst.template}\n{inst.style}")

In [43]:
# inst.nuxt_path[1:]

In [5]:
from dspy import ChainOfThought
from typing import List
from pydantic import Field
from dslmodel import DSLModel


class VueProp(DSLModel):
    """Model for Vue props. Must use javascript types for the `type` field."""
    name: str = Field(..., description="The name of the prop.")
    type: str = Field(...,
                      description="The expected javascript type of the prop. String, Number, Boolean, Array, Object, or Function.")
    default: str | None = Field(None, description="Default value of the prop (optional).")


class VueComponent(DSLModel):
    pascal_name: str = Field(..., description="The PascalCase name of the component.")
    kebab_name: str = Field(..., description="The kebab-case name of the component.")
    template: str = Field("<template>\n</template>", description="<template> block of the Vue component.")
    props: list[VueProp] = Field([], description="List of props for the component.")
    setup: str = Field("<script setup>\n</script>",
                       description="The <script setup> block of the component. No placeholders. Nuxt Core Team quality code only.")
    style: str = Field("<style scoped>\n</style>", description="The <style scoped> block of the component.")
    nuxt_path: str = Field("", description="The path to the component in a Nuxt project.")


from dspy import Signature, InputField, OutputField


class GenerateVueSetupCode(Signature):
    """
    Generate a Nuxt Core Team-level <script setup> block for a Vue component.
    """
    pascal_name = InputField(desc="The PascalCase name of the Vue component.")
    kebab_name = InputField(desc="The kebab-case name of the Vue component.")
    props = InputField(
        desc=(
            "A list of VueProp objects, where each contains 'name' (string), 'type' (string: "
            "one of String, Number, Boolean, Array, Object, Function), and an optional 'default' (string or None)."
        )
    )
    prompt = InputField(desc="The prompt or instructions for the Vue component.")

    setup_code = OutputField(
        desc="The <script setup> block for the Vue component. Must be Nuxt core team quality.")


def generate_setup_code_for_component_with_dspy(component: VueComponent, prompt: str) -> VueComponent:
    """
    Use DSPy's ChainOfThought pipeline with the GenerateVueSetupCode signature
    to generate and attach a Nuxt Core Team-level <script setup> block
    for a VueComponent instance.

    Args:
        component (VueComponent): An instance of the VueComponent model.

    Returns:
        VueComponent: The updated instance with the generated setup code attached.
    """
    # Serialize the VueComponent instance to a dictionary for use with the ChainOfThought pipeline
    input_data = {
        "pascal_name": component.pascal_name,
        "kebab_name": component.kebab_name,
        "props": [
            {"name": prop.name, "type": prop.type, "default": prop.default}
            for prop in component.props
        ],
        "prompt": prompt,
    }

    # Use the DSPy ChainOfThought with the `GenerateVueSetupCode` signature
    result = ChainOfThought(GenerateVueSetupCode)(**input_data)

    # Attach the generated code to the `setup` field of the component
    component.setup = result.setup_code
    return component


instructions = (
    "Hello World. Must include a template that shows 'Hello, {{ name }}!'. Name must be a prop of type String with a default value of 'World'."
    " useFetch to get data from the Weather API.")

vue_comp = VueComponent.from_prompt(instructions)

generate_setup_code_for_component_with_dspy(vue_comp, instructions)

print(vue_comp.setup)

In [7]:
from dslmodel import init_lm

init_lm("groq/llama-3.1-70b-versatile")

generate_setup_code_for_component_with_dspy(vue_comp, instructions)

print(vue_comp.setup)

In [8]:
print(vue_comp)

In [26]:
from dslmodel.utils.file_tools import extract_code
from pydantic import BaseModel, Field
from typing import Dict
from enum import Enum
from dslmodel import render, DSLModel


class VueProp(DSLModel):
    """Model for Vue props. Must use javascript types for the `type` field."""
    name: str = Field(..., description="The name of the prop.")
    type: str = Field(...,
                      description="The expected javascript type of the prop. String, Number, Boolean, Array, Object, or Function.")
    default: str | None = Field(None, description="Default value of the prop (optional).")


class VueComponent(DSLModel):
    pascal_name: str = Field(..., description="The PascalCase name of the component.")
    kebab_name: str = Field(..., description="The kebab-case name of the component.")
    template: str = Field("<template>\n</template>", description="<template> block of the Vue component.")
    props: list[VueProp] = Field([], description="List of props for the component.")
    setup: str = Field("<script setup>\n</script>",
                       description="The <script setup> block of the component. No placeholders. Nuxt Core Team quality code only.")
    style: str = Field("<style scoped>\n</style>", description="The <style scoped> block of the component.")
    nuxt_path: str = Field("", description="The path to the component in a Nuxt project.")

    def write_file(self):
        # If it starts with a slash, remove it
        if self.nuxt_path.startswith("/"):
            self.nuxt_path = self.nuxt_path[1:]

        with open(self.nuxt_path, "w") as f:
            f.write(f"{self.setup}\n{self.template}\n{self.style}")
        print(f"Component written to {self.nuxt_path}")


class GenerateVueTemplate(Signature):
    """
    Generate a Nuxt Core Team-level <template> block for a Vue component.
    Do not include ```vue``` in the output.
    """
    pascal_name = InputField(desc="The PascalCase name of the Vue component.")
    kebab_name = InputField(desc="The kebab-case name of the Vue component.")
    props = InputField(
        desc=(
            "A list of VueProp objects, where each contains 'name' (string), 'type' (string: "
            "one of String, Number, Boolean, Array, Object, Function), and an optional 'default' (string or None)."
        )
    )
    prompt = InputField(
        desc="The prompt or instructions for the Vue component. E.g., its role, layout, or behavior in the UI."
    )

    template_code = OutputField(
        desc="The <template> block for the Vue component. Must be Nuxt core team quality.",
        prefix="<template>\n",
    )


def generate_template_code_for_component_with_dspy(component: VueComponent, prompt: str) -> VueComponent:
    """
    Use DSPy's ChainOfThought pipeline with the GenerateVueTemplate signature
    to generate and attach a Nuxt Core Team-level <template> block
    for a VueComponent instance.

    Args:
        component (VueComponent): An instance of the VueComponent model.

    Returns:
        VueComponent: The updated instance with the generated template code attached.
    """
    # Serialize the VueComponent instance to match the Signature inputs
    input_data = {
        "pascal_name": component.pascal_name,
        "kebab_name": component.kebab_name,
        "props": [
            {"name": prop.name, "type": prop.type, "default": prop.default}
            for prop in component.props
        ],
        "prompt": prompt,
    }

    # Use DSPy ChainOfThought with the `GenerateVueTemplate` signature
    result = ChainOfThought(GenerateVueTemplate)(**input_data)

    # Attach the generated template code to the `template` field of the component
    component.template = result.template_code
    return component


class GenerateVueSetupCode(Signature):
    """
    Generate a Nuxt Core Team-level <script setup> block for a Vue component.
    Do not include ```vue``` in the output.
    """
    pascal_name = InputField(desc="The PascalCase name of the Vue component.")
    kebab_name = InputField(desc="The kebab-case name of the Vue component.")
    props = InputField(
        desc=(
            "A list of VueProp objects, where each contains 'name' (string), 'type' (string: "
            "one of String, Number, Boolean, Array, Object, Function), and an optional 'default' (string or None)."
        )
    )
    prompt = InputField(desc="The prompt or instructions for the Vue component.")

    setup_code = OutputField(
        desc="The <script setup> block for the Vue component. Must be Nuxt core team quality.",
        prefix="```vue\n")


def generate_setup_code_for_component_with_dspy(component: VueComponent, prompt: str) -> VueComponent:
    """
    Use DSPy's ChainOfThought pipeline with the GenerateVueSetupCode signature
    to generate and attach a Nuxt Core Team-level <script setup> block
    for a VueComponent instance.

    Args:
        component (VueComponent): An instance of the VueComponent model.

    Returns:
        VueComponent: The updated instance with the generated setup code attached.
    """
    # Serialize the VueComponent instance to a dictionary for use with the ChainOfThought pipeline
    input_data = {
        "pascal_name": component.pascal_name,
        "kebab_name": component.kebab_name,
        "props": [
            {"name": prop.name, "type": prop.type, "default": prop.default}
            for prop in component.props
        ],
        "prompt": prompt,
    }

    # Use the DSPy ChainOfThought with the `GenerateVueSetupCode` signature
    result = ChainOfThought(GenerateVueSetupCode)(**input_data)

    # Attach the generated code to the `setup` field of the component
    component.setup = extract_code(result.setup_code)
    return component


instructions = (
    "Hello World. Must include a template that shows 'Hello, {{ name }}!'. Name must be a prop of type String with a default value of 'World'."
    " useFetch to get data from the Weather API. Get all values from the useFetch. Include the weather data in the template. Maximum code quality.")

vue_comp = VueComponent.from_prompt(instructions)

generate_setup_code_for_component_with_dspy(vue_comp, instructions)

generate_template_code_for_component_with_dspy(vue_comp, instructions)

vue_comp.write_file()

In [29]:
from dslmodel import DSLModel
from pydantic import BaseModel, Field
from typing import List, Optional


class NuxtQueryParameter(DSLModel):
    """Model for a Nuxt server query parameter."""
    name: str = Field(..., description="The name of the query parameter.")
    type: str = Field(...,
                      description="The JavaScript data type of the query parameter (e.g., string, number, boolean).")
    required: bool = Field(True, description="Whether the query parameter is required.")
    description: str = Field("", description="Description of the query parameter.")


class NuxtResponseField(DSLModel):
    """Model for a Nuxt server response field."""
    name: str = Field(..., description="The name of the response field.")
    type: str = Field(..., description="The JavaScript data type of the response field.")
    description: str = Field("", description="Description of the response field.")


class NuxtServerRoute(DSLModel):
    """Model for defining a Nuxt server route."""
    path: str = Field(..., description="The API endpoint path (e.g., /api/weather).")
    method: str = Field("GET", description="HTTP method (e.g., GET, POST).")
    query_parameters: List[NuxtQueryParameter] = Field([], description="Query parameters for the route.")
    response_fields: List[NuxtResponseField] = Field([], description="Response fields for the API.")
    logic: str = Field(
        "",
        description="JavaScript/TypeScript logic for the route, written as a string of server-side code.",
    )

    def render_code(self) -> str:
        """
        Generate the JavaScript/TypeScript code for the Nuxt server route.

        Returns:
            str: Generated Nuxt server route code.
        """
        # Render query parameters
        query_params = ", ".join(
            f"{param.name}: {param.type}{'' if param.required else ' = undefined'}"
            for param in self.query_parameters
        )

        # Render response fields
        response_data = ",\n    ".join(
            f"{field.name}: <{field.type}> // {field.description}" for field in self.response_fields
        )

        # Render complete route code
        route_code = f"""
export default defineEventHandler(async (event) => {{
    const query = getQuery(event);
    const {{ {query_params} }} = query;

    // Validate query parameters
    {''.join([f'if (!{param.name}) throw createError({{ statusCode: 400, message: "Missing {param.name}" }});\n    ' for param in self.query_parameters if param.required])}

    // Server-side logic
    {self.logic}

    // Mock response (replace with actual logic results)
    return {{
        {response_data}
    }};
}});
"""
        return route_code.strip()


from dslmodel.utils.file_tools import extract_code
from dspy import ChainOfThought, Signature, InputField, OutputField


class GenerateNuxtServerLogic(Signature):
    """
    Generate the server-side logic for a Nuxt H3 server route.
    """
    path = InputField(desc="The API path.")
    query_parameters = InputField(desc="A list of NuxtQueryParameter objects.")
    response_fields = InputField(desc="A list of NuxtResponseField objects.")
    prompt = InputField(desc="Detailed instructions or context for the route's purpose.")

    logic_code = OutputField(desc="The generated logic for the H3 route. Nuxt core team quality code.",
                             prefix="```javascript\n")


def generate_server_logic_with_dspy(route: NuxtServerRoute, prompt: str) -> NuxtServerRoute:
    """
    Use DSPy's ChainOfThought to generate server-side logic for a Nuxt server route.

    Args:
        route (NuxtServerRoute): The Nuxt server route model.
        prompt (str): The prompt or instructions for the logic generation.

    Returns:
        NuxtServerRoute: The updated server route model with generated logic.
    """
    input_data = {
        "path": route.path,
        "query_parameters": [
            {"name": param.name, "type": param.type, "required": param.required, "description": param.description}
            for param in route.query_parameters
        ],
        "response_fields": [
            {"name": field.name, "type": field.type, "description": field.description}
            for field in route.response_fields
        ],
        "prompt": prompt,
    }

    # Call DSPy ChainOfThought with the GenerateNuxtServerLogic signature
    result = ChainOfThought(GenerateNuxtServerLogic)(**input_data)

    # Attach the generated logic to the route
    route.logic = extract_code(result.logic_code)
    return route


# Define a route to create a new pet
create_pet_route = NuxtServerRoute(
    path="/api/pet",
    method="POST",
    query_parameters=[
        NuxtQueryParameter(name="name", type="string", required=True, description="Name of the pet."),
        NuxtQueryParameter(name="species", type="string", required=True, description="Species of the pet."),
        NuxtQueryParameter(name="age", type="number", required=False, description="Age of the pet (optional)."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Name of the pet."),
        NuxtResponseField(name="species", type="string", description="Species of the pet."),
        NuxtResponseField(name="age", type="number", description="Age of the pet."),
    ],
)

# Define a route to get pet details
get_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="GET",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Name of the pet."),
        NuxtResponseField(name="species", type="string", description="Species of the pet."),
        NuxtResponseField(name="age", type="number", description="Age of the pet."),
    ],
)

# Define a route to update pet details
update_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="PUT",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
        NuxtQueryParameter(name="name", type="string", required=False, description="Updated name of the pet."),
        NuxtQueryParameter(name="species", type="string", required=False, description="Updated species of the pet."),
        NuxtQueryParameter(name="age", type="number", required=False, description="Updated age of the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Updated name of the pet."),
        NuxtResponseField(name="species", type="string", description="Updated species of the pet."),
        NuxtResponseField(name="age", type="number", description="Updated age of the pet."),
    ],
)

# Define a route to delete a pet
delete_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="DELETE",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="success", type="boolean", description="Whether the deletion was successful."),
    ],
)

# Example prompt for creating a pet
create_pet_prompt = (
    "Implement the logic to create a new pet in the pet shop. "
    "The pet should have a name, species, and optionally an age. "
    "Generate a unique ID for the pet and return it along with the provided data."
)

generate_server_logic_with_dspy(create_pet_route, create_pet_prompt)

# Example prompt for getting a pet's details
get_pet_prompt = (
    "Implement the logic to retrieve a pet's details by its unique ID. "
    "If the pet does not exist, return a 404 error with an appropriate message."
)

generate_server_logic_with_dspy(get_pet_route, get_pet_prompt)

# Example prompt for updating a pet's details
update_pet_prompt = (
    "Implement the logic to update an existing pet's details by its unique ID. "
    "Allow partial updates. Return the updated pet data, or a 404 error if the pet does not exist."
)

generate_server_logic_with_dspy(update_pet_route, update_pet_prompt)

# Example prompt for deleting a pet
delete_pet_prompt = (
    "Implement the logic to delete a pet by its unique ID. "
    "Return a success field indicating whether the deletion was successful."
)

generate_server_logic_with_dspy(delete_pet_route, delete_pet_prompt)

# Render the CRUD routes
print(create_pet_route.render_code())
# print(get_pet_route.render_code())
# print(update_pet_route.render_code())
# print(delete_pet_route.render_code())


In [31]:
print(get_pet_route.render_code())

In [ ]:
# Define a route to create a new pet
create_pet_route = NuxtServerRoute(
    path="/api/pet",
    method="POST",
    query_parameters=[
        NuxtQueryParameter(name="name", type="string", required=True, description="Name of the pet."),
        NuxtQueryParameter(name="species", type="string", required=True, description="Species of the pet."),
        NuxtQueryParameter(name="age", type="number", required=False, description="Age of the pet (optional)."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Name of the pet."),
        NuxtResponseField(name="species", type="string", description="Species of the pet."),
        NuxtResponseField(name="age", type="number", description="Age of the pet."),
    ],
)

# Define a route to get pet details
get_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="GET",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Name of the pet."),
        NuxtResponseField(name="species", type="string", description="Species of the pet."),
        NuxtResponseField(name="age", type="number", description="Age of the pet."),
    ],
)

# Define a route to update pet details
update_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="PUT",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
        NuxtQueryParameter(name="name", type="string", required=False, description="Updated name of the pet."),
        NuxtQueryParameter(name="species", type="string", required=False, description="Updated species of the pet."),
        NuxtQueryParameter(name="age", type="number", required=False, description="Updated age of the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="id", type="string", description="Unique identifier for the pet."),
        NuxtResponseField(name="name", type="string", description="Updated name of the pet."),
        NuxtResponseField(name="species", type="string", description="Updated species of the pet."),
        NuxtResponseField(name="age", type="number", description="Updated age of the pet."),
    ],
)

# Define a route to delete a pet
delete_pet_route = NuxtServerRoute(
    path="/api/pet/{id}",
    method="DELETE",
    query_parameters=[
        NuxtQueryParameter(name="id", type="string", required=True, description="Unique identifier for the pet."),
    ],
    response_fields=[
        NuxtResponseField(name="success", type="boolean", description="Whether the deletion was successful."),
    ],
)
